In [20]:
# FASTF1 Setup & 2023 Data Export
import fastf1
from fastf1 import get_event_schedule, get_session
import pandas as pd
import os

In [21]:
# Enable cache
cache_dir = '../.fastf1_cache'
os.makedirs(cache_dir, exist_ok=True)
fastf1.Cache.enable_cache(cache_dir) # Stores data locally

In [22]:
# Prepare output folder
output_dir = '../data/processed'
os.makedirs(output_dir, exist_ok=True)

In [23]:
# Get all 2023 race weekends
schedule = get_event_schedule(2023, include_testing=False)
race_events = schedule[schedule['EventFormat'] == 'conventional']

In [24]:
# All races holder
all_races = []

for _, row in race_events.iterrows():
    round_num = row['RoundNumber']
    gp_name = row['EventName'].lower().replace(" ", "_")

    try:
        session = get_session(2023, round_num, 'R')
        session.load()
        laps = session.laps.reset_index(drop=True)

        # Select columns
        selected_cols = [
            'Driver', 'LapNumber', 'LapTime',
            'Sector1Time', 'Sector2Time', 'Sector3Time',
            'Compound', 'TyreLife', 'Stint',
            'PitInTime', 'PitOutTime', 'TrackStatus', 'IsAccurate'
        ]
        lap_data = laps[selected_cols].copy()

        # Convert time columns to seconds
        time_cols = ['LapTime', 'Sector1Time', 'Sector2Time', 'Sector3Time']
        for col in time_cols:
            lap_data[f'{col}Seconds'] = lap_data[col].dt.total_seconds()

        # Drop rows with critical NaNs
        lap_data.dropna(subset=[
            'LapTimeSeconds', 'Sector1TimeSeconds', 'Sector2TimeSeconds', 'Sector3TimeSeconds', 'Compound'
        ], inplace=True)

        # Add GP column
        lap_data['GrandPrix'] = gp_name

        # Export individual file
        race_csv_path = f"{output_dir}/race_summary_2023_{gp_name}.csv"
        lap_data.to_csv(race_csv_path, index=False)
        print(f"✅ Exported: {gp_name}")

        # Add to combined
        all_races.append(lap_data)

    except Exception as e:
        print(f"❌ Failed: {gp_name} — {e}")

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Pri

✅ Exported: bahrain_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 11 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '22', '27', '24', '21', '81', '2', '4', '77', '23', '18']
core           INFO 	Loading data for Australian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_ap

✅ Exported: saudi_arabian_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '44', '14', '18', '11', '4', '27', '81', '24', '22', '77', '55', '10', '31', '21', '2', '20', '63', '23', '16']
core           INFO 	Loading data for Miami Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...


...

✅ Exported: mexico_city_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 1 completed the race distance 00:00.001000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '11', '31', '18', '55', '44', '63', '14', '81', '10', '23', '20', '3', '24', '2', '77', '22', '27', '4']
core           INFO 	Loading data for Abu Dhabi Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_d

✅ Exported: las_vegas_grand_prix


req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '16', '63', '11', '4', '81', '14', '22', '44', '18', '3', '31', '10', '23', '27', '2', '24', '55', '77', '20']


✅ Exported: abu_dhabi_grand_prix


In [25]:
# Combine all
if all_races:
    combined_df = pd.concat(all_races, ignore_index=True)
    combined_csv_path = f"{output_dir}/all_races_combined_with_sectors.csv"
    combined_df.to_csv(combined_csv_path, index=False)
    print(f"\n🎉 Combined export done: {combined_csv_path}")
else:
    print("⚠️ No race data exported successfully.")


🎉 Combined export done: ../data/processed/all_races_combined_with_sectors.csv
